# Nuova sezione

In [ ]:
#Download dataset

!wget  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/5y9wdsg2zt-1.zip -O Crack_dataset.zip


--2021-05-14 17:47:16--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/5y9wdsg2zt-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.36.43
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.36.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240847944 (230M) [application/octet-stream]
Saving to: ‘Crack_dataset.zip’

Crack_dataset.zip   100%[===================>] 229.69M  32.6MB/s    in 7.9s    

2021-05-14 17:47:24 (29.0 MB/s) - ‘Crack_dataset.zip’ saved [240847944/240847944]



In [ ]:
 #Unzip dataset
from zipfile import ZipFile
file_name = "/content/Crack_dataset.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  

!unrar x Concrete\ Crack\ Images\ for\ Classification.rar -inul
print("done")

 





done


In [ ]:
!mkdir Dataset
!mv Negative Dataset/Non-Cracked
!mv Positive Dataset/Cracked
!rm Crack_dataset.zip
!rm Concrete\ Crack\ Images\ for\ Classification.rar

mkdir: cannot create directory ‘Dataset’: File exists


In [ ]:
import os
print(f"Cracked: {len(os.listdir('Dataset/Cracked'))}")
print(f"Non-cracked: {len(os.listdir('Dataset/Non-Cracked'))}")

Cracked: 20001
Non-cracked: 20001


In [ ]:
#load deck dataset
from torchvision.datasets import ImageFolder

dataset = ImageFolder("Dataset")

In [ ]:
#check
from collections import Counter
print(dataset)
print(dataset.imgs)
print(dataset.classes)
print(Counter(dataset.targets))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(dataset.imgs, dataset.targets, test_size=0.2, shuffle=True)


In [ ]:
from torchvision import transforms
transform = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize([0.5] * 3, [0.5] * 3)  #[0.5, 0.5, 0.5]                           
])

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class ImageLoader(Dataset):
  def __init__(self, dataset, transform=None):
    self.dataset = dataset
    self.transform = transform

  def __getitem__(self, item):
    image = Image.open(self.dataset[item][0])
    if self.transform is not None:
      return self.transform(image), self.dataset[item][1]
    return image, self.dataset[item][1]

  def __len__(self):
    return len(self.dataset)

  def checkRGB(self, dataset):
    datasetRGB = []
    for index in range(len(dataset)):
      if Image.open(dataset[index][0]).getbands != ("R", "G", "B"):
        datasetRGB.append(dataset[index])
    return datasetRGB

In [ ]:
#check
img_loader = ImageLoader(train_data)
if len(train_data) == len(img_loader.checkRGB(train_data)):
  print("all images are RGB")

all images are RGB


In [ ]:
#check
img_loader = ImageLoader(train_data, transform)
print(img_loader[0][0].size())

torch.Size([3, 227, 227])


In [ ]:
from torch.utils.data import DataLoader
batch_size = 10
train_loader = DataLoader(ImageLoader(train_data, transform), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ImageLoader(test_data, transform), batch_size=batch_size, shuffle=False)


In [ ]:
#network definition
from torch.nn import Module, Conv2d, Linear, MaxPool2d, AdaptiveAvgPool1d
from torch.nn.functional import relu

class Network(Module):

  def __init__(self):
    super(Network, self).__init__()
    self.conv1 = Conv2d(in_channels=3, out_channels=64, kernel_size=5)
    self.conv2 = Conv2d(in_channels=64, out_channels=128, kernel_size=3)
    self.conv3 = Conv2d(in_channels=128, out_channels=256, kernel_size=5)

    self.pool = MaxPool2d(kernel_size=4)
    self.AAPooling = AdaptiveAvgPool1d(256)

    self.fc1 = Linear(in_features=256, out_features=128)
    self.fc2 = Linear(in_features=128, out_features=64)
    self.out = Linear(in_features=64, out_features=2)

  def forward(self, x):
    x = self.pool(relu(self.conv1(x)))
    x = self.pool(relu(self.conv2(x)))
    x = self.pool(relu(self.conv3(x)))
    
    
    x = x.view(1, x.size()[0], -1)
    x = self.AAPooling(x).squeeze()
    

    x = relu(self.fc1(x))
    x = relu(self.fc2(x))

    
    return self.out(x)
    



In [ ]:
#check
sample_data = iter(train_loader)
sample_images = next(sample_data)

net = Network()
out = net(sample_images[0])
print(out)
sample_images[1]


tensor([[ 0.0807, -0.0362],
        [ 0.0747, -0.0387],
        [ 0.0824, -0.0338],
        [ 0.0798, -0.0350],
        [ 0.0742, -0.0382],
        [ 0.0741, -0.0381],
        [ 0.0814, -0.0346],
        [ 0.0799, -0.0367],
        [ 0.0761, -0.0380],
        [ 0.0803, -0.0357]], grad_fn=<AddmmBackward>)


tensor([0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [ ]:
#Training
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 3
#batch_size = 10
learning_rate = 0.001

#class_weights = torch.tensor([0.85, 0.15]).to(device)

from torch.nn import CrossEntropyLoss, BCELoss

model = Network().to(device)

criterion = CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/3], Step [100/6400], Loss: 0.6720
Epoch [1/3], Step [200/6400], Loss: 0.6814
Epoch [1/3], Step [300/6400], Loss: 0.7086
Epoch [1/3], Step [400/6400], Loss: 0.6969
Epoch [1/3], Step [500/6400], Loss: 0.6792
Epoch [1/3], Step [600/6400], Loss: 0.6851
Epoch [1/3], Step [700/6400], Loss: 0.6921
Epoch [1/3], Step [800/6400], Loss: 0.6908
Epoch [1/3], Step [900/6400], Loss: 0.6882
Epoch [1/3], Step [1000/6400], Loss: 0.6725
Epoch [1/3], Step [1100/6400], Loss: 0.6591
Epoch [1/3], Step [1200/6400], Loss: 0.6727
Epoch [1/3], Step [1300/6400], Loss: 0.6667
Epoch [1/3], Step [1400/6400], Loss: 0.6682
Epoch [1/3], Step [1500/6400], Loss: 0.6549
Epoch [1/3], Step [1600/6400], Loss: 0.6767
Epoch [1/3], Step [1700/6400], Loss: 0.6659
Epoch [1/3], Step [1800/6400], Loss: 0.6629
Epoch [1/3], Step [1900/6400], Loss: 0.6663
Epoch [1/3], Step [2000/6400], Loss: 0.6717
Epoch [1/3], Step [2100/6400], Loss: 0.6598
Epoch [1/3], Step [2200/6400], Loss: 0.6582
Epoch [1/3], Step [2300/6400], Loss: 0.61

In [ ]:
#testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(2):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    classes ={1:"Uncracked", 0:"Cracked"}
    for i in range(2):
       acc = 100.0 * n_class_correct[i] / n_class_samples[i]
       print(f'Accuracy of {classes[i]}: {acc} %')



Accuracy of the network: 97.9625 %
Accuracy of Cracked: 98.64615384615385 %
Accuracy of Uncracked: 97.01587301587301 %


In [ ]:
#testing for overfitting
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(2):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    classes ={1:"Uncracked", 0:"Cracked"}
    for i in range(2):
       acc = 100.0 * n_class_correct[i] / n_class_samples[i]
       print(f'Accuracy of {classes[i]}: {acc} %')

